### Hyper-parameters

In [2]:
emsize = 256 # embedding dimension
nhid = 512 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 6 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 8 # the number of heads in the multiheadattention models
dropout = 0.1  # the dropout value
batch_size = 512
eval_batch_size = 64
epochs = 100 # The number of epochs

### Load and Batch Data

In [3]:
# pip install torchtext==0.6.0
import torch
import torchtext
from torchtext.data.utils import get_tokenizer
TEXT = torchtext.data.Field(tokenize=get_tokenizer("basic_english"),
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)
train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)
%env CUDA_VISIBLE_DEVICES = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ntokens = len(TEXT.vocab.stoi) # the size of vocabulary
def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)



env: CUDA_VISIBLE_DEVICES=3


In [4]:
from transformer_modules import *
train_data = batchify(train_txt, batch_size)
val_data = batchify(val_txt, eval_batch_size)
test_data = batchify(test_txt, eval_batch_size)

bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

In [5]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output = eval_model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

In [6]:
best_val_loss = float("inf")

best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()
torch.save(best_model.state_dict(), 'models/Transformer.pth')

-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 22.28s | valid loss  7.82 | valid ppl  2486.62
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   2 | time: 22.14s | valid loss  7.27 | valid ppl  1433.43
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   3 | time: 22.19s | valid loss  7.98 | valid ppl  2908.34
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   4 | time: 22.65s | valid loss  7.02 | valid ppl  1122.42
--------------------------------------------------------------------------

-----------------------------------------------------------------------------------------
| end of epoch  34 | time: 38.56s | valid loss  5.08 | valid ppl   160.37
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch  35 | time: 34.50s | valid loss  5.07 | valid ppl   158.51
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch  36 | time: 33.38s | valid loss  5.06 | valid ppl   157.83
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch  37 | time: 32.89s | valid loss  5.05 | valid ppl   155.94
--------------------------------------------------------------------------

-----------------------------------------------------------------------------------------
| end of epoch  67 | time: 33.55s | valid loss  4.96 | valid ppl   142.32
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch  68 | time: 34.92s | valid loss  4.96 | valid ppl   142.12
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch  69 | time: 37.86s | valid loss  4.96 | valid ppl   142.08
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch  70 | time: 33.90s | valid loss  4.95 | valid ppl   141.87
--------------------------------------------------------------------------

-----------------------------------------------------------------------------------------
| end of epoch 100 | time: 37.59s | valid loss  4.94 | valid ppl   140.02
-----------------------------------------------------------------------------------------


In [7]:
best_model_loaded = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
best_model_loaded.load_state_dict(torch.load('models/Transformer.pth'), strict=True)
test_loss = evaluate(best_model_loaded, test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

| End of training | test loss  4.86 | test ppl   129.57


# Please provide your analysis on each hyper-parameter.
Double-click on this part and write your content. And run this cell